## IMPORTATION DES BIBLIOTHEQUES ET PARAMETRES GLOBAUX

In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import importlib

%matplotlib qt
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman']
plt.rcParams.update({'font.size': 16})

pd.set_option('display.max_rows', 100)

liste_normalisations=['nombreObs','ObsUnique','nombreObs_normalisé_log',
                      'nombreObs_normalisé_par_espece','nombreObs_normalisé_par_maille',
                      'nombreObs_normalisé_par_maille_classe','nombreObs_normalisé_par_maille_regne',
                      'nombreObs_normalisé_mrl','nombreObs_normalisé_mcl',
                     ]


In [3]:
# Importation des fonctions définies dans les fichiers biodiv

from affichage_carte_biodiv import (
    n_minimum,
    n_maximum,
    afficher_carte_maille,
    afficher_carte_observations,
    afficher_carte_observations_espece_manquante,
    afficher_carte_cluster
)

from normalisation_biodiv import (
    cap_to_nth_largest,
    normalize_cap,
    normalize_maille,
    normalize_maille_clade,
    normalize_espece,
    normalize_clade,
    normalize_unique,
    normalize_log,
    normalize_mcl,
    normalize_mrl
)

from exploration_biodiv import (
    creation_filtre_top,
    top_especes,
    chercher_espece,
    explorer_clade,
    etude_mailles
)

from correlation_biodiv import correlation_sujet_recherche

from biodiversite_endemisme_biodiv import (
    shannon,
    simpson,
    WE,
    calcul_indice
)

from clustering_geo_biodiv import (
    PC_analysis,
    clustering,
    determine_k,
    composition_cluster
)

from clustering_espece_biodiv import (
    correlation_matrix_dendogram,
    dendogram,
    cluster_from_correlation,
    correlation_cluster,
    chercher_cluster_espece,
    afficher_liste_cluster,
    liste_espece_cluster,
    groupe_dans_maille,
    etude_cluster_local,
    chercher_especes_pas_presentes,
    chercher_zone_espece_pas_presente
)

from fonctions_annexes_biodiv import (
    round_to_sig,
    dictionnaire_especes,
    affichage_dataframe
)


## IMPORTATION DES DONNEES

In [ ]:
# Importer directement toutes les données
df_inpn = pd.DataFrame()

path='C:/Users/anormand/Documents/Projet Python/Biodiv'
savepath='C:/Users/anormand/Documents/Projet Python/Biodiv/Résultats'

df_inpn = pd.read_csv(path+'/Data/format2/data_INPN_1980_all.csv',dtype=str)

In [4]:
# Importer les données par classe

# Nom de la classe
# poissons, mammiferes, crustaces, escargots, fonges, reptiles, insectes
classes= ['mammiferes', 'crustaces', 'escargots', 'fonges', 'reptiles', 'insectes','poissons','oiseaux','plantes','autres']

# Création du DataFrame
df_inpn_all = pd.DataFrame()

path='C:/Users/anormand/Documents/Projet Python/Biodiv'
savepath='C:/Users/anormand/Documents/Projet Python/Biodiv/Résultats'

for classe in classes:
    # Charger les données de la classe
    df_temp = pd.read_csv(path+'/Data/format2/data_INPN_1970_' + classe + '.csv',dtype=str)
    df_temp['nombreObs']=df_temp['nombreObs'].astype(int)
    #df_temp['Année']=df_temp['Année'].astype(float)
    #df_temp['Période']=df_temp['Période'].astype(float)
    #df_temp['day_of_year']=df_temp['day_of_year'].astype(float)
    
    # Ajouter le DataFrame à la liste des DataFrames
    df_inpn_all = pd.concat([df_inpn_all, df_temp], ignore_index=True)


In [5]:
# Garder seulement le premier nom Vernaculaire
df_inpn_all['nomVernaculaire'] = df_inpn_all['nomVernaculaire'].str.split(',').str[0]

# filtrer pour garder uniquement les règnes animal et végétal
df_inpn=df_inpn_all[(df_inpn_all['regne']=='Animalia')|(df_inpn_all['regne']=='Plantae')]

# Ajout des données géographiques
# Charger les données géographiques des mailles 10 km
mailles_fp = path+r'\Data\map\L93_10X10.shp'  # Remplace par le chemin vers ton fichier
carte_maille = gpd.read_file(mailles_fp)

nouveaux_noms_columns = {'CD_SIG': 'codeMaille10Km'}
carte_maille = carte_maille.rename(columns=nouveaux_noms_columns)

df_inpn=df_inpn[df_inpn['codeMaille10Km'].isin(carte_maille['codeMaille10Km'])]
#df_geo_inpn=pd.merge(df_inpn,carte_maille,on="codeMaille10Km")

# Normalisations des données
df_inpn=normalize_unique(df_inpn)
df_inpn=normalize_log(df_inpn)
df_inpn=normalize_espece(df_inpn)
df_inpn=normalize_maille(df_inpn)
df_inpn=normalize_maille_clade(df_inpn, code_col='codeMaille10Km', clade_col='classe', observation_col='nombreObs')
df_inpn=normalize_maille_clade(df_inpn, code_col='codeMaille10Km', clade_col='regne', observation_col='nombreObs')
df_inpn=normalize_mrl(df_inpn)
df_inpn=normalize_mcl(df_inpn)

print('filtre des espèces les plus présentes')
df_inpn_top = creation_filtre_top(df_inpn,'nombreObs',100)

Le nombre d observations total par espèce est fixé à 10 000
Il y a en moyenne 13839.4 observations par maille
Il y a en moyenne 1320.9 observations par classe par maille
Il y a en moyenne 8407.2 observations par regne par maille
filtre des espèces les plus présentes
nombre d'espèces retenues dans le df :15162 (32%)


## EXPLORATION DES DONNEES  

In [6]:
# Créer des filtres prédéfinis
print('arbres')
df_arbres=df_inpn[(df_inpn['classe']=='Pinopsida')|(df_inpn['ordre']=='Fagales')|(df_inpn['genre']=='Crataegus')|(df_inpn['genre']=='Prunus')|(df_inpn['genre']=='Malus')|(df_inpn['genre']=='Sorbus')|(df_inpn['genre']=='Pyrus')]
df_arbres = df_arbres[df_arbres['genre'] != 'Juniperus']
df_arbres_top=creation_filtre_top(df_arbres,'nombreObs',100)

print('plantes')
df_plantes=df_inpn[(df_inpn['regne']=='Plantae')]
df_plantes_top=creation_filtre_top(df_plantes,'nombreObs',10000)

print('mammifères')
df_mammifères=df_inpn[(df_inpn['classe']=='Mammalia')]
df_mammifères_top=creation_filtre_top(df_mammifères,'nombreObs',100)

print('poissons')
df_poissons=df_inpn[(df_inpn['classe']=='Actinopterygii')|(df_inpn['classe']=='Elasmobranchii')]
df_poissons_top = creation_filtre_top(df_poissons,'nombreObs',100)

print('oiseaux')
df_oiseaux=df_inpn[(df_inpn['classe']=='Aves')]
df_oiseaux_top= creation_filtre_top(df_oiseaux,'nombreObs',100)

print('reptiles')
df_reptiles=df_inpn[(df_inpn['classe']=='Reptilia')]
df_reptiles_top= creation_filtre_top(df_reptiles,'nombreObs',10)

print('arthropodes')
df_arthropodes=df_inpn[(df_inpn['classe']=='Insecta')|(df_inpn['classe']=='Arachnida')|(df_inpn['classe']=='Malacostraca')|(df_inpn['classe']=='Branchiopoda')|(df_inpn['classe']=='Chilopoda')|(df_inpn['classe']=='Diplopoda')]
df_arthropodes_top = creation_filtre_top(df_arthropodes,'nombreObs',1000)

print('graminees')
df_graminees=df_inpn[(df_inpn['famille']=='Poaceae')]
df_graminees_top = creation_filtre_top(df_graminees,'nombreObs',10000)

print('mousses')
df_mousses=df_inpn[(df_inpn['ordre']=='Sphagnales')|(df_inpn['ordre']=='Hypnales')|
                    (df_inpn['ordre']=='Dicranales')|(df_inpn['ordre']=='Bryales')|
                    (df_inpn['ordre']=='Buxbaumiales')|(df_inpn['ordre']=='Diphysciales')|
                    (df_inpn['ordre']=='Grimmiales')|(df_inpn['ordre']=='Andreaeales')|
                    (df_inpn['ordre']=='Polytrichales')|(df_inpn['ordre']=='Hookeriales')]
df_mousses_top = creation_filtre_top(df_mousses,'nombreObs',100)

"""
# Liste de DataFrames à fusionner
dataframes = [df_arbres_top, df_mammifères_top, df_poissons_top, df_oiseaux_top, df_reptiles_top, df_arthropodes_top, df_inpn_top]

# Fusionner les DataFrames et supprimer les doublons
df_inpn_top = pd.concat(dataframes).drop_duplicates()

# Afficher le DataFrame fusionné
df_inpn_top = creation_filtre_top(df_inpn_top,'nombreObs',1)
"""

arbres
nombre d'espèces retenues dans le df :120 (42%)
plantes
nombre d'espèces retenues dans le df :1128 (10%)
mammifères
nombre d'espèces retenues dans le df :129 (71%)
poissons
nombre d'espèces retenues dans le df :226 (40%)
oiseaux
nombre d'espèces retenues dans le df :469 (67%)
reptiles
nombre d'espèces retenues dans le df :55 (79%)
arthropodes
nombre d'espèces retenues dans le df :1533 (5%)
graminees
nombre d'espèces retenues dans le df :118 (15%)
mousses
nombre d'espèces retenues dans le df :457 (53%)


"\n# Liste de DataFrames à fusionner\ndataframes = [df_arbres_top, df_mammifères_top, df_poissons_top, df_oiseaux_top, df_reptiles_top, df_arthropodes_top, df_inpn_top]\n\n# Fusionner les DataFrames et supprimer les doublons\ndf_inpn_top = pd.concat(dataframes).drop_duplicates()\n\n# Afficher le DataFrame fusionné\ndf_inpn_top = creation_filtre_top(df_inpn_top,'nombreObs',1)\n"

In [6]:
# Afficher le top des espèces les plus observées
print('nombre d"espèces observées :',len(df_inpn['nomScientifique'].unique()))
top_especes(df_inpn,'nombreObs').head(10)

nombre d"espèces observées : 47939


,nomScientifique,nombreObs,nomVernaculaire,genre,famille,ordre,classe,regne,all
0,Pipistrellus pipistrellus,1896908,Pipistrelle commune,Pipistrellus,Vespertilionidae,Chiroptera,Mammalia,Animalia,All
1,Turdus merula,1664186,Merle noir,Turdus,Turdidae,Passeriformes,Aves,Animalia,All
2,Salmo trutta,1614182,Truite de mer,Salmo,Salmonidae,Salmoniformes,Actinopterygii,Animalia,All
3,Fringilla coelebs,1545253,Pinson des arbres,Fringilla,Fringillidae,Passeriformes,Aves,Animalia,All
4,Parus major,1472717,Mésange charbonnière,Parus,Paridae,Passeriformes,Aves,Animalia,All
5,Erithacus rubecula,1256878,Rougegorge familier,Erithacus,Muscicapidae,Passeriformes,Aves,Animalia,All
6,Cyanistes caeruleus,1190753,Mésange bleue,Cyanistes,Paridae,Passeriformes,Aves,Animalia,All
7,Columba palumbus,1148534,Pigeon ramier,Columba,Columbidae,Columbiformes,Aves,Animalia,All
8,Passer domesticus,1055259,Moineau domestique,Passer,Passeridae,Passeriformes,Aves,Animalia,All
9,Sylvia atricapilla,986456,Fauvette à tête noire,Sylvia,Sylviidae,Passeriformes,Aves,Animalia,All


In [7]:
# Chercher les espèces à partir d'un mot 
resultat_recherche=chercher_espece(df_inpn,'Salamandre')
affichage_dataframe(resultat_recherche,['nomScientifique','nomVernaculaire','genre','famille','ordre','classe','regne','nombreObs'],col_sort='nombreObs').head(50)

,nomScientifique,nomVernaculaire,genre,famille,ordre,classe,regne,nombreObs
0,Salamandra salamandra,Salamandre tachetée (La),Salamandra,Salamandridae,Urodela,Amphibia,Animalia,60876
1,Salamandra salamandra terrestris,Salamandre tachetée terrestre (La),Salamandra,Salamandridae,Urodela,Amphibia,Animalia,3152
2,Salamandra corsica,Salamandre de Corse (La),Salamandra,Salamandridae,Urodela,Amphibia,Animalia,557
3,Salamandra lanzai,Salamandre de Lanza (La),Salamandra,Salamandridae,Urodela,Amphibia,Animalia,488
4,Salamandra salamandra fastuosa,Salamandre tachetée fastueuse (La),Salamandra,Salamandridae,Urodela,Amphibia,Animalia,305
5,Salamandra atra,Salamandre noire (La),Salamandra,Salamandridae,Urodela,Amphibia,Animalia,2


In [8]:
# Afficher les sous clades du clade choisi, le nombre d'espèces et le nombre d'observations
# Liste des clades = ['all','regne', 'classe', 'ordre', 'famille', 'genre','nomScientifique','nomVernaculaire']
clade = 'classe'
taxon='Aves'

explorer_clade(df_inpn,clade,taxon)

,ordre,nombreEspèces,nombreObs,Ratio Obs/Esp
0,Passeriformes,267,26897626,100740.2
1,Charadriiformes,134,3928917,29320.3
2,Columbiformes,9,2406193,267354.8
3,Accipitriformes,34,2016809,59317.9
4,Anseriformes,86,1993291,23177.8
5,Pelecaniformes,34,1858189,54652.6
6,Piciformes,11,1202682,109334.7
7,Gruiformes,14,674802,48200.1
8,Falconiformes,12,642636,53553.0
9,Phoenicopteriformes,9,574241,63804.6


In [ ]:
# Afficher la carte avec les mailles
afficher_carte_maille(carte_maille)

In [ ]:
# Spécifier une liste de mailles à étudier
liste_codes=['10kmL93E089N623']
col_choice='nombreObs_normalisé_par_maille_regne'

df_maille=etude_mailles(df_inpn,liste_codes,['nombreObs','nombreObs_normalisé_par_maille_regne','nombreObs_normalisé_par_espece'])
df_dico=dictionnaire_especes(df_inpn)
df_maille=pd.merge(df_maille,df_dico,on='nomScientifique')

affichage_dataframe(df_maille,['nomScientifique','nomVernaculaire','ordre','classe','regne','nombreObs','nombreObs_normalisé_par_maille_regne','nombreObs_normalisé_par_espece'],col_sort=col_choice)

## AFFICHAGE DES DONNEES SOUS FORME DE CARTE 

In [64]:
# Afficher les données sous forme de carte

col_values='nombreObs_normalisé_mrl'
colormap='viridis'

clade = 'nomScientifique' #clade can be :  nomScientifique 	nomVernaculaire  regne 	classe 	ordre 	famille  genre 
taxon='Salvia rosmarinus'
titre=clade+'_'+taxon+' - var '+col_values
df_filt=df_inpn[(df_inpn[clade]==taxon)]

#titre="Mousses"
#df_filt=df_mousses

""" col_values : 
nombreObs  
ObsUnique	
nombreObs_normalisé_par_maille	
nombreObs_normalisé_par_maille_classe
nombreObs_normalisé_mrl
nombreObs_normalisé_log 
nombreObs_normalisé_par_classe 
nombreObs_normalisé_par_espece
"""

""" basemap_type :
'OpenStreetMap': ctx.providers.OpenStreetMap.France,
'CartoDB': ctx.providers.CartoDB.Positron,
'Esri': ctx.providers.Esri.WorldImagery,
'NASAGIBS': ctx.providers.NASAGIBS.BlueMarble,
'GeoportailSatellite': ctx.providers.GeoportailFrance.orthos,
'GeoportailDepartements': ctx.providers.GeoportailFrance.parcels,
'GeoportailRegion': ctx.providers.GeoportailFrance.Cartes_Naturalearth,
"""

afficher_carte_observations(df_filt,carte_maille,col_values,n=1,m=1,cmap_choice=colormap,
                            title=titre,save_path=savepath,basemap_type='OpenStreetMap')

## INDICE DE BIODIVERSITE et ENDEMISME  

In [49]:
# Calculer les indices de biodiversité et d'endémisme
col_values='nombreObs'
code_col='codeMaille10Km'
colormap='plasma'

df_etude=df_inpn_top
df_indice=calcul_indice(df_etude,col_values,code_col)

nom='Tout'

#Indices de biodiversité : shannon ou simpson ou nb espèces
indice_choisi="nombres d'espèces"
titre=nom+' Biodiversité '+' indice '+indice_choisi
afficher_carte_observations(df_etude,carte_maille,'ObsUnique',n=1,m=10,cmap_choice=colormap,
                            title=titre,save_path=savepath,basemap_type='OpenStreetMap')

indice_choisi='shannon' #Mesure la diversité en prenant en compte la richesse spécifique et l'abondance relative des espèces.
titre=nom+' Biodiversité'+' indice '+indice_choisi
afficher_carte_observations(df_indice,carte_maille,indice_choisi,n=100,m=1,cmap_choice=colormap,
                            title=titre,save_path=savepath,basemap_type='OpenStreetMap')

indice_choisi='simpson' ##Mesure la diversité en prenant en compte la richesse spécifique et l'abondance relative des espèces.
titre=nom+' Biodiversité '+' indice '+indice_choisi
afficher_carte_observations(df_indice,carte_maille,indice_choisi,n=100,m=1,cmap_choice=colormap,
                            title=titre,save_path=savepath,basemap_type='OpenStreetMap')

In [ ]:
# Indice d'endémisme : WE
indice_choisi='WE'
titre=nom+' Endémisme'+' indice '+indice_choisi
afficher_carte_observations(df_indice,carte_maille,indice_choisi,n=1,m=50,cmap_choice=colormap,
                            title=titre,save_path=savepath,basemap_type='OpenStreetMap')

## CORRELATIONS AVEC UNE ESPECE  

In [ ]:
# Chercher les correlations entre un sujet et un champ de recherche
clade_sujet = 'nomScientifique'
taxon_sujet='Tarentola mauritanica'

#définition du dataframe où chercher la corrélation
clade_recherche='nomScientifique' #definition du niveau de recherche
df_global=df_inpn
methode='spearman' # pearson kendall spearman
col_values='nombreObs_normalisé_par_maille_regne'
""" col_values : 
nombreObs 
nombreObs_capped
ObsUnique	
nombreObs_normalisé_par_maille	
nombreObs_normalisé_par_maille_regne
"""

sorted_corr=correlation_sujet_recherche(df_inpn,col_values,clade_sujet,taxon_sujet,df_global,clade_recherche,methode)
sorted_corr.head(20)

In [ ]:
# Afficher la carte de l'espèce/autre la mieux corrélée
taxon_corr=sorted_corr.iloc[0, 0]
df_filt=df_global[(df_global[clade_recherche]==taxon_corr)]
titre=clade_recherche+'_'+taxon_corr+'_'+col_values
afficher_carte_observations(df_filt,carte_maille,col_values,n=1,m=10,cmap_choice=colormap,title=titre,save_path=None,basemap_type='OpenStreetMap')

In [ ]:
# Afficher la carte de l'espèce/autre la moins corrélée
taxon_corr=sorted_corr.iloc[len(sorted_corr)-1,0]
df_filt=df_global[(df_global[clade_recherche]==taxon_corr)]
titre=clade_recherche+'_'+taxon_corr+'_'+col_values
afficher_carte_observations(df_filt,carte_maille,col_values,n=1,m=10,cmap_choice=colormap,title=titre,save_path=None,basemap_type='OpenStreetMap')

## ANALYSE PAR CLUSTERS BIOGEOGRAPHIQUES  

In [22]:
# Regrouper les données en clusters similaires et afficher la carte
#clade can be :  nomScientifique 	nomVernaculaire 	regne 	classe 	ordre 	famille 	genre 	
import importlib
import affichage_carte_biodiv
# Recharger le module
importlib.reload(affichage_carte_biodiv)

# Maintenant importer groupe_dans_maille
from affichage_carte_biodiv import afficher_carte_cluster
""" var : 
nombreObs
nombreObs_capped
ObsUnique	
nombreObs_normalisé_par_maille	
nombreObs_normalisé_par_maille_classe  
nombreObs_normalisé_log 
nombreObs_normalisé_par_classe 
nombreObs_normalisé_par_taxon
"""

df_filt=df_inpn
var='nombreObs_normalisé_mrl'
choix_methode='ward' #method : kmeans ward
titre='Carte de France par cluster '+'- var '+var+' - méthode '+choix_methode

"""
clade = 'classe'
taxon='Reptilia'
titre=clade+'_'+taxon+'_'+var
df_filt=df_inpn[(df_inpn[clade]==taxon)]
"""

colormap='Spectral_r' #Spectral_r plasma Paired
nombre_pca=15
k=9

df_pca=PC_analysis(df_filt,col_index='codeMaille10Km',col_values=var,clade='nomScientifique',n_pca=nombre_pca)

#determine_k(df_pca)

df_cluster=clustering(df_pca,col_index='codeMaille10Km',method=choix_methode,k_cluster=k)
"""
    'OpenStreetMap': ctx.providers.OpenStreetMap.France,
    'CartoDB': ctx.providers.CartoDB.Positron,
    'Esri': ctx.providers.Esri.WorldImagery,
    'NASAGIBS': ctx.providers.NASAGIBS.BlueMarble,
    'GeoportailSatellite': ctx.providers.GeoportailFrance.orthos,
    'GeoportailDepartements': ctx.providers.GeoportailFrance.parcels,
    'GeoportailRegion': ctx.providers.GeoportailFrance.Cartes_Naturalearth,
"""
afficher_carte_cluster(df_cluster,carte_maille,col_values="Cluster",cmap_choice=colormap,
                       title=titre,save_path=savepath,basemap_type='GeoportailDepartements')


Variance expliquée par chaque composante: [0.27105072 0.05354412 0.04480471 0.02602863 0.01762808 0.01380339
 0.01299385 0.01126769 0.00956778 0.00859627 0.00833818 0.0079624
 0.00717952 0.00630696 0.00565827]
Cluster
1    3335
2    1880
3    1139
4     968
5     499
6     496
7     392
8     303
9     147
Name: count, dtype: int64


In [23]:
# Afficher les espèces surreprésentés dans chaque cluster
#df_filt=creation_filtre_top(df_inpn,'nombreObs',2000)


df_oiseaux_top= creation_filtre_top(df_inpn,'nombreObs',1000)
cluster_composition=composition_cluster(df_inpn,df_cluster,'nombreObs_normalisé_par_espece',5)

nombre d'espèces retenues dans le df :6083 (13%)
Cluster 1:
Cétochile septentrionale (Le) Calanus finmarchicus        0.1
nan                  Amphissa acutecostata        0.1
nan                  Eunice oerstedii            0.1
nan                  Kuhnia scombri              0.1
nan                  Mohnia abyssorum            0.1


Cluster 2:
Bythinelle de Quenoche Bythinella vesontiana        0.0
Grand hamster        Cricetus cricetus           0.0
nan                  Labiobaetis tricolor        0.0
Ronce canaliculée    Rubus canaliculatus         0.0
Caryer ovale         Carya ovata                 0.0


Cluster 3:
Épinoche de mer      Spinachia spinachia         0.0
Vipérine petit-pin   Echium pininana             0.0
Sélin de Brotero     Selinum broteri             0.0
nan                  Vertebrata lanosa           0.0
nan                  Antennella siliquosa        0.0


Cluster 4:
Muscari de Motelay   Muscari motelayi            0.0
Barbon de Virginie   Andropogon virginic

In [24]:
# Afficher les taxons les plus présents de chaque cluster
cluster_composition=composition_cluster(df_filt,df_cluster,'nombreObs_normalisé_par_maille',5)

Cluster 1:
Fou de Bassan        Morus bassanus             13.7
Goéland brun         Larus fuscus                8.8
Dauphin bleu et blanc Stenella coeruleoalba        7.4
Poisson-lune         Mola mola                   5.0
Rorqual commun       Balaenoptera physalus        4.5


Cluster 2:
Merle noir           Turdus merula               1.4
Pinson des arbres    Fringilla coelebs           1.3
Mésange charbonnière Parus major                 1.2
Chevesne commun      Squalius cephalus           1.1
Pigeon ramier        Columba palumbus            1.0


Cluster 3:
Truite de mer        Salmo trutta                2.6
Saumon de l'Atlantique Salmo salar                 2.1
Pinson des arbres    Fringilla coelebs           1.7
Pipistrelle commune  Pipistrellus pipistrellus        1.7
Merle noir           Turdus merula               1.7


Cluster 4:
Goujon               Gobio gobio                 1.4
Merle noir           Turdus merula               1.3
Mésange charbonnière Parus major       

## ANALYSE PAR CLUSTERS D'ESPECES

In [72]:
# Filtrer les espèces les plus présentes pour réduire le temps de calcul
df_input=creation_filtre_top(df_inpn,'nombreObs',100)

# info temps : seuil=10 000 - n=1680 => t=60s
# pour toutes les espèces (16 792) => t=6000s 
# Cas général : t = n^2 / 47040 ou n=racine(t*47040)

nombre d'espèces retenues dans le df :15162 (32%)


In [28]:
# Calculer la matrice de corrélation et affichage du dendogram
col_values_mrl='nombreObs_normalisé_mrl'
corr_mrl=correlation_matrix_dendogram(df_input,col_values_mrl)
Z_mrl=dendogram(corr_mrl,methode='ward',display=0) #plutot ward ou complete

"""
col_values_obs='nombreObs'
corr_obs=correlation_matrix_dendogram(df_input,col_values_obs)
Z_obs=dendogram(corr_obs,methode='ward',display=0)

col_values_mr='nombreObs_normalisé_par_maille_regne'
corr_mr=correlation_matrix_dendogram(df_input,col_values_mr)
Z_mr=dendogram(corr_mr,methode='ward',display=0) #plutot ward ou complete

col_values_unique='ObsUnique'
corr_unique=correlation_matrix_dendogram(df_input,col_values_unique)
Z_unique=dendogram(corr_unique,methode='ward',display=0)

corr_moy=(corr_mr+corr_unique+corr_obs)/3
Z_moy=dendogram(corr_moy,methode='ward',display=0)
"""

"\ncol_values_unique='ObsUnique'\ncorr_unique=correlation_matrix_dendogram(df_input,col_values_unique)\nZ_unique=dendogram(corr_unique,methode='ward',display=0)\n\ncorr_moy=(corr_mr+corr_unique+corr_obs)/3\nZ_moy=dendogram(corr_moy,methode='ward',display=0)\n"

In [ ]:
# Sauvegarder les données
path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data_intermed'

df_tosave = pd.DataFrame(Z_obs)
df_tosave.to_csv(path+'/Z_mrl_ward_sansfungi.csv', index=False)


In [ ]:
# Importer les données
path='C:/Users/anormand/Documents/Projet Python/Biodiv/Data_intermed'

Z_obs = pd.read_csv(path+ '/Z_obs_sansfungi.csv',dtype=str) # ward

In [30]:
# Former les clusters en fonction du niveau choisi
col_values='nombreObs'
Z=Z_obs
lvl=8
criterion='distance'

df_corr_cluster_obs = cluster_from_correlation(df_input, Z_obs, var=col_values, level=lvl, crit='distance')
df_corr_cluster_mrl = cluster_from_correlation(df_input, Z_mrl_ward, var=col_values, level=lvl, crit='distance')
df_corr_cluster_mr = cluster_from_correlation(df_input, Z_mr_ward, var=col_values, level=lvl, crit='distance')

print('nombre de cluster formés :'+str(len(df_corr_cluster_obs['Cluster_corr'].unique())))
print('nombre de cluster formés :'+str(len(df_corr_cluster_mrl['Cluster_corr'].unique())))
print('nombre de cluster formés :'+str(len(df_corr_cluster_mr['Cluster_corr'].unique())))

nombre de cluster formés :119
nombre de cluster formés :131
nombre de cluster formés :93


In [42]:
# Remplir le dataframe df_inpn_cluster_espece en faisant correspondre maille et cluster

col_values='nombreObs'
df_inpn_cluster_espece=groupe_dans_maille(df_input,df_corr_cluster,col_values)


In [71]:
# Chercher le cluster contenant une espèce en particulier et afficher les espèces qu'il contient, ordonne selon la colonne col_values
espece='Castor fiber'
col_choice='nombreObs_normalisé_par_maille_regne'
df_corr_cluster=df_corr_cluster_mrl
#Recherche du cluster contenant l'espèce
num_cluster=chercher_cluster_espece(df_corr_cluster,df_input,col_choice,espece)

# Affichage du cluster ordonné par col_values
liste_espece_cluster_choisi=liste_espece_cluster(df_input,df_corr_cluster,num_cluster)

print('Le cluster regroupe '+str(len(liste_espece_cluster_choisi))+' espèces')

affichage_dataframe(liste_espece_cluster_choisi,['nomScientifique','nomVernaculaire','ordre','classe','regne','nombreObs','nombreObs_normalisé_par_maille_regne'],col_sort=col_choice).head(100)

Le cluster contenant Castor fiber est le cluster n° 128
Le cluster regroupe 11 espèces


,nomScientifique,nomVernaculaire,ordre,classe,regne,nombreObs,nombreObs_normalisé_par_maille_regne
0,Alburnoides bipunctatus,Spirlin,Cypriniformes,Actinopterygii,Animalia,222363,155033.1
1,Rhodeus amarus,Bouvière,Cypriniformes,Actinopterygii,Animalia,127562,78882.7
2,Blicca bjoerkna,Brème bordelière,Cypriniformes,Actinopterygii,Animalia,68035,39619.7
3,Chondrostoma nasus,Nase commun,Cypriniformes,Actinopterygii,Animalia,62092,36256.8
4,Pseudorasbora parva,Pseudorasbora,Cypriniformes,Actinopterygii,Animalia,51876,31583.5
5,Castor fiber,Castor d'Eurasie,Rodentia,Mammalia,Animalia,33613,21853.3
6,Ameiurus melas,Poisson-chat,Siluriformes,Actinopterygii,Animalia,28604,21037.8
7,Gymnocephalus cernua,Grémille,Perciformes,Actinopterygii,Animalia,23986,16024.1
8,Silurus glanis,Silure glane,Siluriformes,Actinopterygii,Animalia,19140,10243.7
9,Sander lucioperca,Sandre,Perciformes,Actinopterygii,Animalia,10340,7355.1


In [55]:
# Afficher l'aire de répartition correspondant au cluster selectionné

titre=f'{col_choice} cluster numéro {num_cluster}'
colormap='viridis'
col_affichage='nombreObs_normalisé_mrl'
num_cluster=chercher_cluster_espece(df_corr_cluster,df_input,col_choice,espece)

df_cluster_espece=df_inpn_cluster_espece[df_inpn_cluster_espece['Cluster_corr']==num_cluster]
df_cluster_espece = df_cluster_espece.sort_values(by=col_choice,ascending=False)

# On récupère la liste unique des 'codeMaille10Km'

seuil=max(df_cluster_espece[col_affichage])/10
df_filt=df_cluster_espece[df_cluster_espece[col_affichage]>=seuil]

afficher_carte_observations(df_filt,carte_maille,col_values=col_affichage,cmap_choice=colormap,n=1,m=1,
                       title=titre,save_path=savepath,basemap_type='OpenStreetMap')

Le cluster contenant Sus scrofa est le cluster n° 6


## ANALYSE AVANCEE PAR CLUSTER D'ESPECE

In [ ]:
# Afficher la carte avec les mailles
afficher_carte_maille(carte_maille)

In [56]:
liste_geo_PN_Calanques=['10kmL93E088N624','10kmL93E090N624',
             '10kmL93E087N623','10kmL93E088N623','10kmL93E089N623','10kmL93E090N623','10kmL93E091N623',
             '10kmL93E086N622','10kmL93E087N622','10kmL93E088N622','10kmL93E089N622','10kmL93E090N622','10kmL93E091N622',
             '10kmL93E091N620','10kmL93E087N622','10kmL93E088N621','10kmL93E089N621','10kmL93E090N621','10kmL93E091N621',
            ]
liste_geo_PNR_Vosges=['10kmL93E100N680','10kmL93E101N680',
             '10kmL93E100N679','10kmL93E101N679','10kmL93E098N678','10kmL93E099N678','10kmL93E100N678','10kmL93E101N678',
             '10kmL93E098N677','10kmL93E099N677','10kmL93E100N677','10kmL93E101N677',
             '10kmL93E098N676','10kmL93E099N676','10kmL93E100N676','10kmL93E101N676',
             '10kmL93E098N675','10kmL93E099N675','10kmL93E100N675','10kmL93E097N675',
            ]
liste_geo_PN_Cevennes=['10kmL93E073N632', '10kmL93E073N633', '10kmL93E073N634', '10kmL93E073N635', '10kmL93E073N636', '10kmL93E073N637',
            '10kmL93E074N632', '10kmL93E074N633', '10kmL93E074N634', '10kmL93E074N635', '10kmL93E074N636', '10kmL93E074N637',
            '10kmL93E075N632', '10kmL93E075N633', '10kmL93E075N634', '10kmL93E075N635', '10kmL93E075N636', '10kmL93E075N637',
            '10kmL93E076N632', '10kmL93E076N633', '10kmL93E076N634', '10kmL93E076N635', '10kmL93E076N636', '10kmL93E076N637',
            '10kmL93E077N632', '10kmL93E077N633', '10kmL93E077N634', '10kmL93E077N635', '10kmL93E077N636', '10kmL93E077N637',
            '10kmL93E078N635','10kmL93E079N635','10kmL93E079N636','10kmL93E078N636'
            ]

In [68]:
# Rechercher les clusters présents dans une maille ou une liste de mailles, les ordonner selon la colonne col_values
col_choice='nombreObs_normalisé_par_espece' # pour les espèces les plus caractéristiques
liste_codes=liste_geo_PNR_Vosges

df_cluster_maille=etude_mailles(df_inpn_cluster_espece,liste_codes,['nombreObs','nombreObs_normalisé_par_maille_regne','nombreObs_normalisé_par_espece'],col_groupe='Cluster_corr')
df_dico=df_inpn_cluster_espece[['Cluster_corr', 'nomScientifique', 'nomVernaculaire']].drop_duplicates(subset=['Cluster_corr'])
df_cluster_maille = pd.merge(df_cluster_maille,df_dico, on='Cluster_corr', how='left')
df_cluster_maille = df_cluster_maille.sort_values(by=col_choice,ascending=False)
df_cluster_maille=df_cluster_maille.reset_index()

print(str(len(df_cluster_maille))+' groupes sont présents dans la zone étudiée')
affichage_dataframe(df_cluster_maille,['Cluster_corr','nomScientifique','nomVernaculaire','nombreObs_normalisé_par_espece','nombreObs','nombreObs_normalisé_par_maille_regne'],col_sort=col_choice).head(20)

125 groupes sont présents dans la zone étudiée


,Cluster_corr,nomScientifique,nomVernaculaire,nombreObs_normalisé_par_espece,nombreObs,nombreObs_normalisé_par_maille_regne
0,35,"Struthiopteris spicant, Digitalis purpurea sub...","Struthioptéride en épi, Digitale pourpre, Gail...",340.3,518.8,412.2
1,76,"Vaccinium myrtillus, Nardus stricta, Carex nig...","Airelle myrtille, Nard raide, Laîche noire, Bi...",256.5,640.5,443.5
2,80,"Gentiana lutea, Geranium sylvaticum, Prenanthe...","Gentiane jaune, Géranium des bois, Prénanthe p...",237.1,484.9,305.7
3,75,"Drosera rotundifolia, Carex rostrata, Carex ec...","Rossolis à feuilles rondes, Laîche rostrée, La...",215.8,384.5,249.5
4,34,"Oxalis acetosella, Avenella flexuosa, Avenella...","Oxalide petite-oseille, Canche flexueuse, Sorb...",192.0,558.5,431.2
5,125,"Xyleborinus saxesenii, Salpingus planirostris,...",", Taupin acajou, Taupin brun velouté",167.7,234.4,208.8
6,21,"Abies alba, Picea abies, Poecile montanus, Gal...","Sapin blanc, Épicéa commun, Mésange boréale, G...",146.3,611.1,469.0
7,16,"Filipendula ulmaria, Betula pendula, Cirsium p...","Reine-des-prés, Bouleau pleureur, Cirse des ma...",104.4,547.1,412.9
8,25,"Rana temporaria, Caltha palustris, Aphantopus ...","Grenouille rousse (La), Populage des marais, T...",101.9,299.9,222.4
9,28,"Salmo trutta, Cervus elaphus, Carex panicea, R...","Truite de mer, Cerf élaphe, Laîche panic, Pati...",98.5,1245.7,1110.4


In [ ]:
# Chercher le rang du cluster choisi dans la zone selectionnée
num_cluster=290
rank_cluster = df_cluster_maille[df_cluster_maille['Cluster_corr'] == num_cluster].index[0]
print(('Cluster rang ')+str(rank_cluster)+' dans la zone '+str(liste_codes))


In [70]:
# Afficher les espèces présentes dans le cluster le plus représenté localement, 0 pour les espèces du cluster pas présentes localement
col_choice='nombreObs_normalisé_par_espece'

rank_cluster=1 #correspond au numéro du cluster : 0 = le premier

num_cluster=int(df_cluster_maille['Cluster_corr'].iloc[rank_cluster])
print(('Cluster n°')+str(num_cluster)+' dans la zone '+str(liste_codes))
cluster_local=etude_cluster_local(df_input,liste_codes,df_corr_cluster,num_cluster,col_choice)
affichage_dataframe(cluster_local,['nomScientifique','nomVernaculaire','classe','regne','nombreObs','nombreObs_normalisé_par_maille_regne','nombreObs_normalisé_par_espece',col_values],col_sort=col_choice)

Cluster n°76 dans la zone ['10kmL93E100N680', '10kmL93E101N680', '10kmL93E100N679', '10kmL93E101N679', '10kmL93E098N678', '10kmL93E099N678', '10kmL93E100N678', '10kmL93E101N678', '10kmL93E098N677', '10kmL93E099N677', '10kmL93E100N677', '10kmL93E101N677', '10kmL93E098N676', '10kmL93E099N676', '10kmL93E100N676', '10kmL93E101N676', '10kmL93E098N675', '10kmL93E099N675', '10kmL93E100N675', '10kmL93E097N675']


,nomScientifique,nomVernaculaire,classe,regne,nombreObs,nombreObs_normalisé_par_maille_regne,nombreObs_normalisé_par_espece,nombreObs_normalisé_mrl
0,Poa chaixii,Pâturin de Chaix,Equisetopsida,Plantae,615,372.7,470.8,133.5
1,Chaerophyllum hirsutum,Cerfeuil hérissé,Equisetopsida,Plantae,462,316.6,378.4,133.6
2,Bistorta officinalis,Bistorte,Equisetopsida,Plantae,994,620.1,371.6,142.8
3,Crepis paludosa,Crépide des marais,Equisetopsida,Plantae,409,251.3,353.1,126.6
4,Sambucus racemosa,Sureau à grappes,Equisetopsida,Plantae,556,441.3,334.4,141.7
5,Vaccinium myrtillus,Airelle myrtille,Equisetopsida,Plantae,2885,2215.2,303.8,172.4
6,Ranunculus aconitifolius,Renoncule à feuilles d'aconit,Equisetopsida,Plantae,411,282.0,288.4,126.2
7,Asplenium septentrionale,Doradille du Nord,Equisetopsida,Plantae,198,131.6,172.1,111.7
8,Maianthemum bifolium,Maïanthème à deux feuilles,Equisetopsida,Plantae,145,106.5,135.7,105.3
9,Carex nigra,Laîche noire,Equisetopsida,Plantae,404,226.2,121.1,113.2


In [ ]:
# Chercher les espèces qui pourraient potentiellement être présente dans la zone étudiée
# la liste est ordonnée de manière décroissante (espèces les plus adaptées en premier)

df_espece_pas_presentes=chercher_especes_pas_presentes(df_input,df_cluster_maille,df_corr_cluster,liste_codes)
affichage_dataframe(df_espece_pas_presentes,['nomScientifique','nomVernaculaire','genre','famille','ordre','classe','regne','Cluster_corr','nombreObs']).head(20)

In [ ]:
# chercher les zones les plus adaptées à une espèce mais ou elle ne se trouve pourtant pas
espece='Lynx lynx'

col_affichage='nombreObs_normalisé_mrl'
colormap='plasma'

df_cluster_espece_manquante=chercher_zone_espece_pas_presente(df_input,df_corr_cluster,df_inpn_cluster_espece,espece,
                                      col_choice='nombreObs_normalisé_par_maille_regne')

# On groupe par 'codeMaille10Km' et on filtre ceux qui n'ont pas l'espèce
df_avec_espece = df_input[(df_input['nomScientifique']==espece)]

# On récupère la liste unique des 'codeMaille10Km'
liste_maille_avec_espece = df_avec_espece['codeMaille10Km'].unique()

df_resultat_avec=df_cluster_espece_manquante[df_cluster_espece_manquante['codeMaille10Km'].isin(liste_maille_avec_espece)]

# Afficher l'aire de répartition correspondant au cluster selectionné
seuil=max(df_resultat_avec[col_affichage])-3
seuil=0
df_filt=df_cluster_espece_manquante[df_cluster_espece_manquante[col_affichage]>=seuil]

titre='Aire de répartition potentielle de '+espece

afficher_carte_observations_espece_manquante(df_filt,df_resultat_avec,carte_maille,col_values=col_affichage,cmap_choice=colormap,color_present='black',
                                             n=1,m=1,title=titre,save_path=savepath,basemap_type='OpenStreetMap',zoom_map=7)

In [ ]:
#choix des clusters à afficher sous forme de carte

liste_cluster=[13]
#liste_cluster=list(range(1, 101))
seuil =0
var='nombreObs_normalisé_mrl'
df_filt=df_corr_cluster[df_corr_cluster['Cluster_corr'].isin(liste_cluster)]

df_filt=pd.merge(df_in,df_filt[['nomScientifique','Cluster_corr']],on='nomScientifique')
df_filt=df_filt[df_filt[var]>=seuil]
df_filt=df_filt[['codeMaille10Km','Cluster_corr']]
df_filt=df_filt.drop_duplicates()

afficher_carte_cluster(df_filt,carte_maille,col_values="Cluster_corr",cmap_choice=colormap,
                       title=titre,save_path=savepath,basemap_type='OpenStreetMap',val_alpha=0.5)